In [1]:
!pip install pymysql # mysql database connection library

     |████████████████████████████████| 43 kB 1.2 MB/s 


In [202]:
import pymysql.cursors  # For Request Query String
import pandas as pd # Querying results to make dataframe(to divide datas)

conn = pymysql.connect(host='csrd.cksngv0eixsu.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='qwerty1!', \
db='before-pp', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)        # mysql connect inform

In [203]:
cur = conn.cursor() # open to query
sql = "SELECT sg_name, sg_artists FROM bf_sg;"  # select data in DB
cur.execute(sql)    # excute query

109480

In [204]:
result = cur.fetchall() # merging result
conn.close()    # DB disconnect
print(result)   # Check Status
# An error appeared, but it doesn't matter because just too many search results.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [205]:
df = pd.DataFrame(result)   # 전체 음악리스트 확인(playlist_id 기준이기 때문에 중복값 들어있음)
df

,sg_name,sg_artists
0,If (Lp Ver.),Bread
1,All By Myself,Eric Carmen
2,Time After Time,Cyndi Lauper
3,Nothing's Gonna Change My Love For You,George Benson
4,Reality,"Vladimir Cosma, Richard Sanderson"
...,...,...
109475,Spectre,Rob Simonsen
109476,Debussy : Prelude A L'apres-Midi D'un Faune L....,Tal & Groethuysen
109477,Yesterday,The Piano Guys
109478,Coeur,Rob Simonsen


In [206]:
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  # 중복 처리
df

,sg_name,sg_artists
0,If (Lp Ver.),Bread
1,All By Myself,Eric Carmen
2,Time After Time,Cyndi Lauper
3,Nothing's Gonna Change My Love For You,George Benson
4,Reality,"Vladimir Cosma, Richard Sanderson"
...,...,...
43033,Spectre,Rob Simonsen
43034,Debussy : Prelude A L'apres-Midi D'un Faune L....,Tal & Groethuysen
43035,Yesterday,The Piano Guys
43036,Coeur,Rob Simonsen


In [207]:
df1 = df[0:10]  # 너무 많은 양의 데이터를 처리하면 Youtube에서 IP ban을 하기 때문과 테스트를 위해 슬라이싱 
df1

,sg_name,sg_artists
0,If (Lp Ver.),Bread
1,All By Myself,Eric Carmen
2,Time After Time,Cyndi Lauper
3,Nothing's Gonna Change My Love For You,George Benson
4,Reality,"Vladimir Cosma, Richard Sanderson"
5,Raindrops Keep Falling on My Head,B.J. Thomas
6,Take On Me,A-ha
7,September,"Earth, Wind & Fire"
8,(They Long To Be) Close To You (Album Ver.) (드...,The Carpenters
9,Let It Be (Remastered 2015),The Beatles


In [8]:
ext_lists = df1.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

In [9]:
!python3 -m pip install --upgrade "git+https://github.com/nficano/pytube.git"   # pytube-12.0.0 설치

  Cloning https://github.com/nficano/pytube.git to /tmp/pip-req-build-gmjbn_7g
  Running command git clone -q https://github.com/nficano/pytube.git /tmp/pip-req-build-gmjbn_7g
  Created wheel for pytube: filename=pytube-12.0.0-py3-none-any.whl size=56526 sha256=035677e6edf1e66d94d045d5b087293ecbebc02e26ef009cc59d65191073bf86
  Stored in directory: /tmp/pip-ephem-wheel-cache-t28t_7b2/wheels/f1/b5/78/12b4d0c5846b07ac841f094923d3b270c10c0a522dad8c2951
Successfully built pytube


In [10]:
import pytube
from pytube import Search
from pytube import YouTube

audio_lists = []    # 검색결과가 나온 비디오 넣는 리스트
forbidden_lists = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여

for ext in ext_lists:
    try:
        StrClean = "-".join(ext)    # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
        yt = Search(StrClean).results[0]    # Search 메소드를 사용하여 검색하고, 그 결과의 1번째 결과 사용(빈도 상, 1 ~ 2번째 영상에 가장 관련있는 영상 표시)
    except:
        print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
        forbidden_lists.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
    audio_lists.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
    # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달

In [11]:
audio_lists, len(audio_lists)   # 성공 목록 확인 및 개수 확인

([{'If (Lp Ver.)-Bread': <pytube.__main__.YouTube object: videoId=SQGSR88WZtw>},
  {'All By Myself-Eric Carmen': <pytube.__main__.YouTube object: videoId=iN9CjAfo5n0>},
  {'Time After Time-Cyndi Lauper': <pytube.__main__.YouTube object: videoId=VdQY7BusJNU>},
  {"Nothing's Gonna Change My Love For You-George Benson": <pytube.__main__.YouTube object: videoId=Tr97MQiqW38>},
  {'Reality-Vladimir Cosma, Richard Sanderson': <pytube.__main__.YouTube object: videoId=5zq0_x77aAE>},
  {'Raindrops Keep Falling on My Head-B.J. Thomas': <pytube.__main__.YouTube object: videoId=sySlY1XKlhM>},
  {'Take On Me-A-ha': <pytube.__main__.YouTube object: videoId=djV11Xbc914>},
  {'September-Earth, Wind & Fire': <pytube.__main__.YouTube object: videoId=Gs069dndIYk>},
  {"(They Long To Be) Close To You (Album Ver.) (드라마 '그녀는 예뻤다' 삽입곡)-The Carpenters": <pytube.__main__.YouTube object: videoId=fBuNYi0Bdjw>},
  {'Let It Be (Remastered 2015)-The Beatles': <pytube.__main__.YouTube object: videoId=HzvDofigTKQ>}],


In [12]:
forbidden_lists, len(forbidden_lists)   # 실패 목록 확인 및 개수 확인

[]

In [47]:
keys = list(audio_lists[0].items())[0][0]   #   딕셔너리 언패킹을 위한 자료형 확인용
link = audio_lists[0].get(keys)
keys, link
type(keys), type(link)

(str, pytube.__main__.YouTube)

In [248]:
from pytube.cli import on_progress
import os

unfounded_lists = []    # 검색된 영상 중 필터링된 관련성 떨어지는 목록용 리스트

def youtube_download(var):
    keys = str(list(var.items())[0][0])  # 딕셔너리에서 파일 이름으로 쓸 string 언패킹
    link = var.get(keys)            # 딕셔너리에서 파일이름이 key값이므로, values 값인  (pytube.__main__.YouTube) 자료형 언패킹
    audio_streams = link.streams.filter(only_audio=True)    # pytube에서 지원하는 다양한 stream 중 Audio 속성을 가진 stream만
    print(audio_streams)  # audio streams 처리 전체 정보

    # 조건문은 나가있어 뒤지기 싫으면
    flt1a = (int(link.length)) > 130    # 영상 길이 2분 10초 초과
    flt1b = (int(link.length)) < 150    # 영상 길이 10분 미만
    flt1 = flt1a and flt1b              # 위의 두 조건 동시에 만족하는 필터
    flt2 = ("official" in str(link.title).lower() or "audio" in str(link.title).lower() or "lylic" in str(link.title).lower())  
    # "official", "audio", "lylic" 중 제목에 하나라도 들어가는 영상 제목 통과

    # 내용 체크
    print(f'영상제목: {link.title},  영상 조회수: {link.views},  영상 길이: {link.length} sec. [{str(link.length // 60).zfill(2)}:{str(link.length % 60).zfill(2)}]')
    print(f'영상 URL: {link.watch_url}')
    print(f"# 1번 체크포인트 - 음원 길이(초):  {int(link.length)}, 이놈의 타입은? : {type(int(link.length))}")
    print(f"# 2번 체크포인트 - 음원 제목(소문자):  {str(link.title).lower()}") 
    print(f"# 3번 체크포인트 - 2분 이상 10분 이하 길이:  {flt1}  ,  음원에 자주 붙은 문자열 포함 여부: {flt2}")

    ## filter()를 활용하여 파일 확장자 타입 등을 포함한 streams 처리
    if flt1 or flt2 == True:    # 위의 1, 2 조건 중 한가지라도 만족한다면 다운로드
        audio_streams = link.streams.filter(file_extension='mp4').get_by_itag(140)  # 여러개의 다운로드 스트림 선택해서 링크 하나 가져오기
        # audio_streams - 예시: <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
        # print(audio_streams) # 가능한 stream 표시
        
        # 저장 폴더 만들기 / Local 혹은 Server 환경에서 돌릴 경우 사용
        # if not os.path.exists('aurigin'):
        #     os.mkdir('aurigin')
        # else:
        #     pass

        title = keys  # 영상 제목으로 파일명이 하고 싶다면? audio_streams.title 또는 link.title 사용

        # special_char = '\/:*?"<>|.'
        # for c in special_char:
        #     if c in title:
        #         title = title.replace(c, '')
        # print(title)  # 파일명에 혹시라도 특수문자 들어갈까봐 Regex마냥 사용할 수 있는거.. 왜 근데 이사람은 정규식 안 썼지?
    
        audio_streams.download(filename= title + '.mp3', output_path='test')     # 설정한 파일명, 경로에 다운로드
        # print(f"=============== {keys.spilt('-')[1]}란 가수의 {keys.spilt('-')[0]} 란 노래 음원 다운로드 완료! ===============")    # 성공 로그
        print(f"#@#@#@#@#@#@#@ {keys} 다운로드 성공!")             # 성공 로그
        print("============================================== 구분선 ==============================================")
    else:
        print(f"^^^^^^^^^^^ Related Video Not Found : {keys}")  # 실패 로그
        unfounded_lists.append(keys)    # 실패 목록 리스트에 저장
        print("============================================== 구분선 ==============================================")

In [249]:
for audio in audio_lists:
    try:
        youtube_download(audio)
    except:
        print(f"no sibal keep going, {keys}")

[<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">, <Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="250" mime_type="audio/webm" abr="70kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">]
영상제목: Bread - If,  영상 조회수: 4423,  영상 길이: 203 sec. [03:23]
영상 URL: https://youtube.com/watch?v=SQGSR88WZtw
# 1번 체크포인트 - 음원 길이(초):  203, 이놈의 타입은? : <class 'int'>
# 2번 체크포인트 - 음원 제목(소문자):  bread - if
# 3번 체크포인트 - 2분 이상 10분 이하 길이:  False  ,  음원에 자주 붙은 문자열 포함 여부: False
^^^^^^^^^^^ Related Video Not Found : If (Lp Ver.)-Bread
============================================== 구분선 ==============================================
[<Stream: itag="139" mime_type="audio/mp4" a

In [253]:
def unfounded_df(ufs):   # unfounded_lists를 DB에 적용하기 위한 Dataframe 만들기
    df = pd.DataFrame(ufs, columns = ['col'])
    df['sg_name'] = df.col.str.split('-').str[0]
    df['sg_artists'] = df.col.str.split('-').str[1]
    df.drop(['col'], axis=1, inplace=True)
    return df

In [254]:
unfounded_df(unfounded_lists)   # 데이터프레임 확인

,sg_name,sg_artists
0,If (Lp Ver.),Bread
1,Nothing's Gonna Change My Love For You,George Benson
2,Reality,"Vladimir Cosma, Richard Sanderson"
3,Raindrops Keep Falling on My Head,B.J. Thomas
4,(They Long To Be) Close To You (Album Ver.) (드...,The Carpenters
5,Let It Be (Remastered 2015),The Beatles


In [223]:
!rm -rf *

In [224]:
!ls